In [13]:
from matplotlib import pyplot as plt
from nltk.tokenize import sent_tokenize
import pandas as pd
import re
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')
import numpy as np
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity
from sklearn.metrics.pairwise import cosine_similarity
import semanticscholar as sch
from IPython.display import display, Latex, HTML, FileLink
from sentence_transformers import SentenceTransformer, models
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

In [2]:
summ=pd.read_csv('sentences.csv')
df=summ.head(100)
df.head()

,task,paper_id,doi,title,summary,score,sentences
0,what is the immune system response to 2019-ncov ?,40dabecefc2f0a2b750e000d1c15cca57a3f7dd4,10.1016/0163-7258(85)90033-6,ANIMAL VIRUSES OF ECONOMIC IMPORTANCE: GENETIC...,in broad terms the immune system among vertebr...,0.927862,[('in broad terms the immune system among vert...
1,what is the immune system response to 2019-ncov ?,2008f024274c53b0de1276f4047a6a641813fd98,10.1016/S1081-1206(10)61294-X,ABSTRACTS: CONCURRENT SESSIONS,a defect of the innate immune system more part...,0.764587,[('a defect of the innate immune system more p...
2,what is the immune system response to 2019-ncov ?,6aadca94314fe7e81e278fbbef178f2a5bf4f538,10.1186/cc13531,Minimising prescribing errors in the ICU DJ Me...,also in these patients cortisol correlates to ...,0.763463,[('also in these patients cortisol correlates ...
3,what is the immune system response to 2019-ncov ?,fea50730d7cdcd7c728f98c31fe20aeb8af5e350,10.1016/j.compbiomed.2020.103670,Computers and viral diseases. Preliminary bioi...,cysteine c should be avoided perhaps replaced ...,0.744909,[('cysteine c should be avoided perhaps replac...
4,what is the immune system response to 2019-ncov ?,68ac63121bded12e1db3178a0c9b050154f81ab8,10.1016/j.mehy.2015.11.005,Seasonality and selective trends in viral acut...,although heat stress may boost the immune syst...,0.724500,[('although heat stress may boost the immune s...


In [3]:
def postprocessing(text):
    # capitalize the text
    text = text.capitalize()
    # '2 , 3' -> '2,3'
    text = re.sub(r'(\d) \, (\d)', "\g<1>,\g<2>", text)
    # full stop
    text = re.sub(r' (\.|\!|\?) (\w)', lambda pat: pat.groups()[0]+" "+pat.groups()[1].upper(), text)
    # full stop at the end
    text = re.sub(r' (\.|\!|\?)$', "\g<1>", text)
    # comma
    text = re.sub(r' (\,|\;|\:) (\w)', "\g<1> \g<2>", text)
    # - | \ / @ and _ (e.g. 2 - 3  -> 2-3)
    text = re.sub(r'(\w) (\-|\||\/|\\|\@\_) (\w)', "\g<1>\g<2>\g<3>", text)
    # parenthesis
    text = re.sub(r'(\(|\[|\{) ([^\(\)]+) (\)|\]|\})', "\g<1>\g<2>\g<3>", text)
    # "" e.g. " word "  -> "word"
    text = re.sub(r'(\") ([^\"]+) (\")', "\g<1>\g<2>\g<3>", text)
    # apostrophe  e.g. l ' a  -> l'a
    text = re.sub(r'(\w) (\') (\w)', "\g<1>\g<2>\g<3>", text)
    # '3 %' ->  '3%'
    text = re.sub(r'(\d) \%', "\g<1>%", text)
    # '# word'  -> '#word'
    text = re.sub(r'\# (\w)', "#\g<1>", text)
    # https and doi
    text = re.sub(r'(https|doi) : ', "\g<1>:", text)
    return text

In [4]:
def remove_stopwords(sen):     
    sen_new = " ".join([i for i in sen if i not in stop_words])          
    return sen_new

In [5]:
def checkAnyStop(token_list, token_stops):
    return any([stop in token_list for stop in token_stops])

def firstFullStopIdx(token_list, token_stops):
    """
    Returns the index of first full-stop token appearing.  
    """
    idxs = []
    for stop in token_stops:
        if stop in token_list:
            idxs.append(token_list.index(stop))
    minIdx = min(idxs) if idxs else None
    return minIdx

puncts = ['!', '.', '?', ';']
puncts_tokens = [tokenizer.tokenize(x)[0] for x in puncts]

def splitTokens(tokens, punct_tokens, split_length):
    """
    To avoid splitting a sentence and lose the semantic meaning of it, a paper is splitted 
    into chunks in such a way that each chunk ends with a full-stop token (['.' ';' '?' or '!']) 
    """
    splitted_tokens = []
    while len(tokens) > 0:
        if len(tokens) < split_length or not checkAnyStop(tokens, punct_tokens):
            splitted_tokens.append(tokens)
            break
        # to not have too long parapraphs, the nearest fullstop is searched both in the previous 
        # and the next strings.
        prev_stop_idx = firstFullStopIdx(tokens[:split_length][::-1], puncts_tokens)
        next_stop_idx = firstFullStopIdx(tokens[split_length:], puncts_tokens)
        if pd.isna(next_stop_idx):
            splitted_tokens.append(tokens[:split_length - prev_stop_idx])
            tokens = tokens[split_length - prev_stop_idx:]
        elif pd.isna(prev_stop_idx):
            splitted_tokens.append(tokens[:split_length + next_stop_idx + 1])
            tokens = tokens[split_length + next_stop_idx + 1:] 
        elif prev_stop_idx < next_stop_idx:
            splitted_tokens.append(tokens[:split_length - prev_stop_idx])
            tokens = tokens[split_length - prev_stop_idx:]
        else:
            splitted_tokens.append(tokens[:split_length + next_stop_idx + 1])
            tokens = tokens[split_length + next_stop_idx + 1:] 
    return splitted_tokens

def splitParagraph(text, split_length=90):
    tokens = tokenizer.tokenize(text)
    splitted_tokens = splitTokens(tokens, puncts_tokens, split_length)
    return [tokenizer.convert_tokens_to_string(x) for x in splitted_tokens]

In [7]:
i=1
sentences = [] 
for s in df['summary']: 
    
              
        for x in splitParagraph(s) :
            sentences.append(x)
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ") 
        # make alphabets lowercase 

clean_sentences = [s.lower() for s in clean_sentences]
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences ]



In [8]:

# Extract word vectors 
word_embeddings = {} 
f = open(r"C:\meriem\Mémoire\glove6b\glove.6B.100d.txt", encoding='utf-8') 
for line in f: 
    values = line.split() 
    word = values[0] 
    coefs = np.asarray(values[1:], dtype='float32')    
    word_embeddings[word] = coefs 
f.close()

In [9]:
sentence_vectors = [] 
for i in clean_sentences: 
    if len(i) != 0: 
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001) 
    else: 
        v = np.zeros((100,)) 
    sentence_vectors.append(v)

In [10]:
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)): 
      for j in range(len(sentences)): 
        if i != j: 
              sim_mat[i][j] = cosine_similarity (sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [11]:
import networkx as nx 
nx_graph = nx.from_numpy_array(sim_mat) 
scores = nx.pagerank(nx_graph)

In [17]:
#pour 50 summary input 
ranked_sentences = sorted(((scores[i],s) for i,s in 
                           enumerate(sentences)), reverse=True)
# Extract top 10 sentences as the summary 
for i in range(23): 
      print(postprocessing(ranked_sentences[i][1]))

The innate immune response is initiated by host proteins called pattern recognition receptors prrs that sense pathogen-associated molecular patterns pamps presented during the microbial infection. The adaptive immune response to rv infection has been well characterized and studies have demonstrated that cd8 + t cells play a critical role in the timely clearance of a primary infection while b cells are crucial for resistance to re-infection. Approximately 21 hr later cell lysates were made and analyzed using the dual-luciferase reporter assay system.
However for several reasons many virus vector systems are still limited in their ability to induce a broad and long-lasting antiviral immune response capable to prevent hiv infection and/or to reduce viral load. An important prerequisite for the initiation of immune responses in vivo is the translocation of antigens from peripheral sites into secondary lymphoid organs zinkernagel et al 1997.
Though the exact cellular targets for these acces